## Hybrid Search in MyScale

This guide introduces the benefits of using hybrid search to improve your text search experience and provides instructions on implementing it with MyScale.

You need to provide your MyScale Cluster connection information within the first code block.

In [32]:
import time
import uuid
import clickhouse_connect
from clickhouse_connect.driver.client import Client
from prettytable import PrettyTable
from sentence_transformers import SentenceTransformer

# MyScale connection information.
host = "your cluster end_point"
port = 443
username = "your username"
password = "your password"
database = "default"
table = "wiki_abstract_50w"
dataset_rows = 500000
dataset_url = "https://myscale-example-datasets.s3.amazonaws.com/wiki_abstract_with_vector_50w.parquet"


# get_client function is used to get a MyScale client.
def get_client(_host: str, _port: int, _username: str, _password: str) -> Client:
    return clickhouse_connect.get_client(host=_host, port=_port, user=_username, password=_password,
                                         session_id=str(uuid.uuid4()), send_receive_timeout=30)

# Print your content in table view.
def print_results(result_rows, field_names):
    x = PrettyTable()
    x.field_names = field_names
    for row in result_rows:
        x.add_row(row)
    x.set_style(13)
    print(x)

# Initialize MyScale client.
client = get_client(host, port, username, password)

# Use transformer all-MiniLM-L6-v2
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

We have improved the [Wikipedia abstract dataset](https://myscale-example-datasets.s3.amazonaws.com/wiki_abstract_with_vector_50w.parquet) hosted by RedisSearch by including vector data. We used `sentence-transformers/all-MiniLM-L6-v2` to convert the text in the `body` column into 384-dimensional vectors. These vectors are stored in the `body_vector` column, and cosine is used to calculate the distance between them.

First, we need to create a table.

In [33]:
# Create a table.
client.command(f"DROP TABLE IF EXISTS {database}.{table} sync;")
client.command(f"""CREATE TABLE {database}.{table}
(
    `id` UInt64,
    `body` String,
    `title` String,
    `url` String,
    `body_vector` Array(Float32),
    CONSTRAINT check_length CHECK length(body_vector) = 384
)
ENGINE = MergeTree
ORDER BY id;""")
print(f"Tables in cluster:{client.query('SHOW tables').result_rows}")

Tables in cluster:[('rd0',), ('wiki_abstract_50w',)]


⏫ Then, we will upload the dataset to the MyScale Cluster.

In [34]:
# Upload data from S3.
time_upload_data_begin = time.time()
try:
    print(f"Start uploading data from S3 to MyScale.")
    client.command(
        f"INSERT INTO {database}.{table} SELECT * FROM s3('{dataset_url}','Parquet');")
except Exception as e:
    print("Upload data from S3 to MyScale may need more time.")
    _client = get_client(host, port, username, password)
    while True:
        rows_count = _client.query(f"SELECT count(*) from {database}.{table}").result_rows[0][0]
        if rows_count >= dataset_rows:
            print("\nData has been uploaded completely.")
            break
        else:
            print(f"\rRows in Table:{rows_count}, time consume:{(time.time() - time_upload_data_begin):.2f} sec.",
                  end='', flush=True)
        time.sleep(1)
print(f"Total upload time consume:{(time.time() - time_upload_data_begin):.2f} sec.")

print(f"Rows in Table:{client.query(f'SELECT count(*) from {database}.{table}').result_rows[0][0]}")


Start uploading data from S3 to MyScale.


Unexpected Http Driver Exception


Upload data from S3 to MyScale may need more time.
Rows in Table:466944, time consume:35.30 sec.
Data has been uploaded completely.
Total upload time consume:36.53 sec.
Rows in Table:500000


We will create a vector index to accelerate vector search speed.

In [35]:
# Create a vector index.
time_build_index_begin = time.time()
client.command(f"OPTIMIZE TABLE {database}.{table} FINAL;")
client.command(
    f"ALTER TABLE {database}.{table} ADD VECTOR INDEX WIKI_MSTG body_vector TYPE MSTG('metric_type=Cosine');")
while True:
    try:
        status = client.query(
            f"SELECT status FROM system.vector_indices "
            f"WHERE CASE WHEN database = '{database}' AND table = '{table}' THEN 1 ELSE 0 END").result_rows[0][0]
        print(f"\rBuilding vector index, status is {status}, "
            f"time consume:{(time.time() - time_build_index_begin):.2f} sec.", end='', flush=True)
        if status == 'Built':
            break
        time.sleep(1)
    except Exception as e:
        print(f"Exception happened when getting vector index build status, {e}")
print(f"\nTotal index build time consume:{(time.time() - time_build_index_begin):.2f} sec.")


Building vector index, status is Built, time consume:64.23 sec. sec.
Total index build time consume:64.24 sec.


It is clear that using a pure vector search for short-text phrases has limitations.

For instance: let's convert the phrase "BGLE Island" into a vector, conduct a vector search, and look at the results.

In [36]:
# Hybrid Search
terms = "BGLE Island"
terms_embedding = model.encode([terms])[0]
terms_pattern = [f'(?i){x}' for x in terms.split(' ')]

# Stage 1. Vector Recall
stage1 = f"""
SELECT id, title, body, distance('alpha=1') (body_vector,{list(terms_embedding)}) AS distance FROM {database}.{table}
ORDER BY distance ASC LIMIT 200"""

stage1_result = client.query(query=stage1)
print_results(stage1_result.result_rows[:5], ["ID", "Title", "Body", "vector_distance"])


|   ID   |          Title          |                        Body                        |  vector_distance   |
|:------:|:-----------------------:|:--------------------------------------------------:|:------------------:|
| 356292 |  O'Brien Island, Chile  |                    Brien Island                    | 0.4032055139541626 |
| 393545 |    Gorgon gas project   |              | place = Barrow Island               | 0.4499066472053528 |
| 123268 | Geography of Montserrat |        | archipelago      = Leeward Islands        | 0.4716024398803711 |
| 127502 |       Saint Kitts       |          | archipelago = Leeward Islands           | 0.4716024398803711 |
| 17716  |        Baltic Sea       | (slightly east of the north tip of Gotland Island) | 0.4798816442489624 |


Let's use a hybrid search to improve result accuracy instead of relying solely on vector search for shorter phrases or single words. For example, for the term "`BGLE Island`", we will follow a `two-stage` approach:
- Use a vector search to identify the top `200` candidates
- Use MyScale's built-in functions and a simplified `TF-IDF` (term frequency-inverse document frequency) method to reorganize and refine these results


In [37]:

# Stage 2. Term Reranking
stage2 = f"""
SELECT tempt.id, tempt.title,tempt.body, FQ, TF_IDF FROM ({stage1}) tempt
ORDER BY length(multiMatchAllIndices(arrayStringConcat([body, title], ' '), {terms_pattern})) AS FQ DESC,
log(1 + countMatches(arrayStringConcat([title, body], ' '), '(?i)({terms.replace(' ', '|')})')) AS TF_IDF DESC limit 10
"""

stage2_result = client.query(query=stage2)
print_results(stage2_result.result_rows, ["ID", "Title", "Body", "FQ", "TF_IDF"])

|   ID   |          Title           |                                                                                                                                                             Body                                                                                                                                                             | FQ |       TF_IDF       |
|:------:|:------------------------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|:--:|:------------------:|
| 50978  |      Trump Islands       |                                     Trump Islands () is a small group of islands lying 4 nautical miles (7 km) southwest of Dodman Island, off the west coast of Graham Land. The islands were disco